In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import nltk_libs

In [ ]:
import simplejson as json
import urllib
from urllib.request import urlopen

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
from __future__ import print_function
import geopandas as gpd
import os
import zipfile
import requests
import fnmatch
import shapely.wkt
#import simplejson as json
#from urllib.request import urlopen

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from wordcloud import WordCloud
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.tokenize import word_tokenize
from collections import Counter
from PIL import Image, ImageDraw, ImageFont
from nltk.util import ngrams 

In [ ]:
df_enc = pd.read_csv('https://raw.githubusercontent.com/lulidezan97/dashboard_demo/master/bases/df_cuestionario.csv')

# Reporte encuesta online sobre la cuarentena en Argentina

In [ ]:
df_enc = df_enc.loc[(df_enc.edad!='Menor de 18 años')]

In [ ]:
edades = ['Entre 18 y 25 años', 'Entre 26 y 35 años', 'Entre 36 y 45 años', 'Entre 46 y 55 años',
          'Entre 56 y 65 años', 'Más de 66 años']

In [ ]:
y_mujeres = [df_enc.loc[(df_enc.edad==i) & (df_enc.genero=='Mujer')]['genero'].count() for i in edades]
y_hombres = [df_enc.loc[(df_enc.edad==i) & (df_enc.genero=='Hombre')]['genero'].count() for i in edades]

fig = go.Figure()
fig.add_trace(go.Bar(x=edades,
                     y=y_mujeres,
                name='Mujeres',
                marker_color='rgb(255, 153, 55)'
                ))
fig.add_trace(go.Bar(x=edades,
                y=y_hombres,
                name='Hombres',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Cantidad de encuestados por rango etario y género',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='',
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)

In [ ]:
with urlopen('https://raw.githubusercontent.com/lulidezan97/dashboard_demo/master/bases/provincia.json') as response:
    provincias = json.load(response)
for i in range(len(provincias['features'])):
    provincias['features'][i]['id']=provincias['features'][i]['properties']['in1']

In [ ]:
gdf = pd.read_csv('https://raw.githubusercontent.com/lulidezan97/dashboard_demo/master/bases/dfmapa.csv')

In [ ]:
gdf['id']=['70','06','02','82','14','66','46','18','78','54','26','22','50','74','42','30','38','62','94','34','58','10','86']

In [ ]:
fig = px.choropleth_mapbox(gdf, geojson=provincias, locations='id', color='freq_porubi',
                           color_continuous_scale=["red", "green"],
                           range_color=(0, 700),
                           zoom=3, center = {"lat": -38.416097, "lon": -63.616672},
                           labels={'Respuestas':'freq_porubi'}
                          )
fig.update_layout(title='Encuestados por provincia', margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
genero=['Mujeres','Hombres']
opiniones=['La cuarentena ya no es nada necesaria', 'La cuarentena ya no es tan necesaria',
       'La cuarentena sigue siendo algo necesaria','La cuarentena sigue siendo muy necesaria']
data= [[df_enc.loc[(df_enc.edad==i) & (df_enc.opinion==j)]['opinion'].count() for i in edades]for j in opiniones]

In [ ]:
fig = px.imshow(data,
                labels=dict(x="", y="Opinión", color="Frecuencia"),
                x=edades,
                y=opiniones
               )
fig.update_layout(
    title='Opiniones sobre la cuarentena según rango etario')
fig.show()

In [ ]:
opiniones_circu_ori=['Estoy muy de acuerdo con que se pueda volver a circular entre provincias',
                     'Estoy algo de acuerdo con que se pueda volver a circular entre provincias',
                     'Estoy algo en desacuerdo con que se pueda volver a circular entre provincias',
                     'Estoy muy en desacuerdo con que se pueda volver a circular entre provincias']
opiniones_circu=['Muy de acuerdo', 'Algo de acuerdo', 'En desacuerdo', 'Muy en desacuerdo']

In [ ]:
data= [[df_enc.loc[(df_enc.edad==i) & (df_enc.circulacion==j)]['opinion'].count() for i in edades]for j in opiniones_circu_ori]

In [ ]:
fig = px.imshow(data,
                labels=dict(x="", y="Opinión", color="Frecuencia"),
                x=edades,
                y=opiniones_circu
               )
fig.update_layout(
    title='¿Estás de acuerdo con que se pueda volver a circular entre provincias?')
fig.show()

In [ ]:
# 'Si, pude hacerlo online': 'Pudo hacer sus actividades online', 
#                               'No, tuve que hacerlo de manera presencial': 'Tuvo que hacer sus actividades de manera presencial',
#                               'No, todavía no pude volver a hacerlo': 'No pudo hacer sus actividades',
#                              'No trabajo ni estudio': 'No trabaja ni estudia'

In [ ]:
df_words = pd.read_csv('https://raw.githubusercontent.com/meyersebastian/idle/master/version1407.csv',encoding='utf-8')
opinion_circulacion = df_words.loc[df_words.circulacion_abierta.notnull(),'circulacion_abierta']

In [ ]:
def limpiar(x):
    x=x.lower()
    x=x.replace('\n','')
    x=x.replace('á','a')
    x=x.replace('é','e')
    x=x.replace('í','i')
    x=x.replace('ó','o')
    x=x.replace('ú','u')
    x = x.strip(' ')
    x = ' '.join(x.split('\\n'))
    x = ' '.join(x.split('\n'))
    for signo in list(string.punctuation):
        x = ' '.join(x.split(signo))
    x = x.replace('  ',' ')
    return x

In [ ]:
opinion_circulacion=pd.DataFrame(opinion_circulacion)

In [ ]:
opinion_circulacion['Limpia'] = [limpiar(x) for x in opinion_circulacion.circulacion_abierta]

In [ ]:
stop=stopwords.words('spanish')

In [ ]:
values = ','.join(str(v) for v in opinion_circulacion['Limpia'])
word_tokens = word_tokenize(values)
word_tokens = [w.strip(' ') for w in word_tokens]
filtered_sentence = [w for w in word_tokens if not w in stop]
wc=WordCloud(background_color="white",collocations=True).generate(' '.join(filtered_sentence))
plt.imshow(wc, interpolation="bilinear")
#plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title('Circulación entre provincias')
plt.rcParams['figure.figsize'] = [20,20]
plt.show()

# Opinión sobre la cuarentena en Argentina

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lulidezan97/dashboard_demo/master/bases/femasc_ponderado.csv')

In [ ]:
df['porcentaje'] = df['porcentaje']*100

In [ ]:
df_1 = df.loc[(df.pregunta=='home office')]
df_2 = df.loc[(df.pregunta=='opinion cuarentena')]
df_3 = df.loc[(df.pregunta=='opinion circulacion')]

In [ ]:
df= df.replace({'respuesta': {'La cuarentena sigue siendo algo necesaria':'Sigue siendo algo necesaria',
                               'La cuarentena sigue siendo muy necesaria': 'Sigue siendo muy necesaria',
                               'La cuarentena ya no es nada necesaria': 'Ya no es nada necesaria',
                               'La cuarentena ya no es tan necesaria': 'Ya no es tan necesaria',
                               'Estoy algo de acuerdo con que se pueda volver a circular entre provincias': 'Algo de acuerdo',
                               'Estoy algo en desacuerdo con que se pueda volver a circular entre provincias': 'Algo en desacuerdo',
                               'Estoy muy de acuerdo con que se pueda volver a circular entre provincias': 'Estoy muy de acuerdo',
                               'Estoy muy en desacuerdo con que se pueda volver a circular entre provincias': 'Estoy muy en desacuerdo'
                              }})

In [ ]:
edades_buttons = widgets.RadioButtons(
    description='Rango etario:',
    value='Todas las edades',
    options=['Entre 18 y 25 años','Entre 26 y 35 años','Entre 36 y 45 años', 
             'Entre 46 y 55 años', 'Entre 56 y 65 años','Más de 66 años', 'Todas las edades']
)
genero_buttons = widgets.Dropdown(
    value='Todos',
    options=['Hombre', 'Mujer', 'Todos'],
    description='Género:',
)

container = widgets.HBox(children=[edades_buttons, genero_buttons])

In [ ]:
y_labels2=['Sigue siendo muy necesaria', 'Sigue siendo algo necesaria', 'Ya no es tan necesaria','Ya no es nada necesaria']
y_labels3=['Estoy muy de acuerdo', 'Algo de acuerdo', 'Algo en desacuerdo','Estoy muy en desacuerdo']

y_labels=['Pudo hacer sus actividades online', 'Tuvo que hacer sus actividades de manera presencial',
         'No pudo hacer sus actividades', 'No trabaja ni estudia']
x_labels=[]
for y_label in y_labels:
    x_labels.append(df_1.loc[(df_1['respuesta']==y_label)])
x_labels
x_labels=[sum(values) for values in df_1['respuesta']==y_label for y_label in y_labels]

In [ ]:
#y_labels=['Pudo hacer sus actividades online', 'Tuvo que hacer sus actividades de manera presencial',
#         'No pudo hacer sus actividades', 'No trabaja ni estudia']
y_labels=list(df_1.respuesta.unique())
x_values=list(df_1.groupby(['respuesta']).sum()['porcentaje'])
texto=[round(x,2) for x in x_values]
total_labels = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y_labels, texto)]
data = go.Bar(y=df_1.respuesta,x=df_1.porcentaje, orientation="h")
layout = go.Layout(title='Actividades durante la cuarentena: ¿Pudiste continuar con tus actividades?',  
                   yaxis={'categoryorder':'array', 'categoryarray':['','','','']}, 
                   annotations=total_labels)
figure = go.Figure(data=data, layout=layout)
g_1 = go.FigureWidget(figure)

In [ ]:
y_labels2=list(df_2.respuesta.unique())
x_values2=list(df_2.groupby(['respuesta']).sum()['porcentaje'])
texto2=[round(x,2) for x in x_values2]
total_labels2 = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y_labels2, texto2)]
data2 = go.Bar(y=df_2.respuesta,x=df_2.porcentaje, orientation="h")
layout2 = go.Layout(title='Opinión sobre la frase: La cuarentena sigue siendo necesaria', annotations=total_labels2)
figure2 = go.Figure(data=data2, layout=layout2)
g_2 = go.FigureWidget(figure2)

In [ ]:
y_labels3=list(df_3.respuesta.unique())
x_values3=list(df_3.groupby(['respuesta']).sum()['porcentaje'])
texto3=[round(x,2) for x in x_values3]
total_labels3 = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y_labels3, texto3)]
data3 = go.Bar(y=df_3.respuesta,x=df_3.porcentaje, orientation="h")
layout3 = go.Layout(title='Opinión sobre la frase: Se debería abrir la circulación entre provincias', annotations=total_labels3)
figure3 = go.Figure(data=data3, layout=layout3)
g_3 = go.FigureWidget(figure3)

In [ ]:
# Assign an empty figure widget


# fig_2 = go.Bar(y=df_2.respuesta,x=df_2.porcentaje, orientation="h")
# fig_3 = go.Bar(y=df_3.respuesta,x=df_3.porcentaje, orientation="h")
# g_2 = go.FigureWidget(data=[fig_2],
#                     layout=go.Layout(
#                         title=dict(
#                             text=f"Opinión sobre la cuarentena"
#                         )))
# g_3 = go.FigureWidget(data=[fig_3],
#                     layout=go.Layout(
#                         title=dict(
#                             text=f"Opinión sobre apertura de la circulación entre provincias"
#                         )))

In [ ]:
def response(change):
    if edades_buttons.value!='Todas las edades' and genero_buttons.value!='Todos':
        df_1 = df.loc[(df.pregunta=='home office') & (df.edad==edades_buttons.value) & (df.genero==genero_buttons.value)]
        y1=list(df_1.respuesta.unique())
        x_tot = df_1['porcentaje'].sum()
        x1=list(df_1.groupby(['respuesta']).sum()['porcentaje'])
        x1=[i*100/x_tot for i in x1]
        texto=[round(i,2) for i in x1]
        total_labels = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y1, texto)]

        df_2 = df.loc[(df.pregunta=='opinion cuarentena') & (df.edad==edades_buttons.value) & (df.genero==genero_buttons.value)]
        y2=list(df_2.respuesta.unique())
        x_tot2 = df_2['porcentaje'].sum()
        x2=list(df_2.groupby(['respuesta']).sum()['porcentaje'])
        x2=[i*100/x_tot2 for i in x2]
        texto2=[round(i,2) for i in x2]
        total_labels2 = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y2, texto2)]

        df_3 = df.loc[(df.pregunta=='opinion circulacion') & (df.edad==edades_buttons.value) & (df.genero==genero_buttons.value)]
        y3=list(df_3.respuesta.unique())
        x_tot3 = df_3['porcentaje'].sum()
        x3=list(df_3.groupby(['respuesta']).sum()['porcentaje'])
        x3=[i*100/x_tot3 for i in x3]
        texto3=[round(i,2) for i in x3]
        total_labels3 = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y3, texto3)]

    elif edades_buttons.value=='Todas las edades' and genero_buttons.value!='Todos':
        df_1 = df.loc[(df.pregunta=='home office') & (df.genero==genero_buttons.value)]
        y1=list(df_1.respuesta.unique())
        x_tot = df_1['porcentaje'].sum()
        x1=list(df_1.groupby(['respuesta']).sum()['porcentaje'])
        x1=[i*100/x_tot for i in x1]
        texto=[round(i,2) for i in x1]
        total_labels = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y1, texto)]

        df_2 = df.loc[(df.pregunta=='opinion cuarentena') & (df.genero==genero_buttons.value)]
        y2=list(df_2.respuesta.unique())
        x_tot2 = df_2['porcentaje'].sum()
        x2=list(df_2.groupby(['respuesta']).sum()['porcentaje'])
        x2=[i*100/x_tot2 for i in x2]
        texto2=[round(i,2) for i in x2]
        total_labels2 = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y2, texto2)]

        df_3 = df.loc[(df.pregunta=='opinion circulacion') & (df.genero==genero_buttons.value)]
        y3 = list(df_3.respuesta.unique())
        x_tot3 = df_3['porcentaje'].sum()
        x3=list(df_3.groupby(['respuesta']).sum()['porcentaje'])
        x3=[i*100/x_tot3 for i in x3]
        texto3=[round(i,2) for i in x3]
        total_labels3 = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y3, texto3)]

    elif edades_buttons.value!='Todas las edades' and genero_buttons.value=='Todos':
        df_1 = df.loc[(df.pregunta=='home office') & (df.edad==edades_buttons.value)]
        y1 = list(df_1.respuesta.unique())
        x_tot = df_1['porcentaje'].sum()
        x1=list(df_1.groupby(['respuesta']).sum()['porcentaje'])
        x1=[i*100/x_tot for i in x1]
        texto=[round(i,2) for i in x1]
        total_labels = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y1, texto)]

        df_2 = df.loc[(df.pregunta=='opinion cuarentena') & (df.edad==edades_buttons.value)]
        y2 =list(df_2.respuesta.unique())
        x_tot2 = df_2['porcentaje'].sum()
        x2=list(df_2.groupby(['respuesta']).sum()['porcentaje'])
        x2=[i*100/x_tot2 for i in x2]
        texto2=[round(i,2) for i in x2]
        total_labels2 = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y2, texto2)]

        df_3 = df.loc[(df.pregunta=='opinion circulacion') & (df.edad==edades_buttons.value)]
        y3 =list(df_3.respuesta.unique())
        x_tot3 = df_3['porcentaje'].sum()
        x3=list(df_3.groupby(['respuesta']).sum()['porcentaje'])
        x3=[i*100/x_tot3 for i in x3]
        texto3=[round(i,2) for i in x3]
        total_labels3 = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y3, texto3)]

    else:
        df_1 = df.loc[(df.pregunta=='home office')]
        y1 =list(df_1.respuesta.unique())
        x_tot = df_1['porcentaje'].sum()
        x1=list(df_1.groupby(['respuesta']).sum()['porcentaje'])
        x1=[i*100/x_tot for i in x1]
        texto=[round(i,2) for i in x1]
        total_labels = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y1, texto)]

        df_2 = df.loc[(df.pregunta=='opinion cuarentena')]
        y2 =list(df_2.respuesta.unique())
        x_tot2 = df_2['porcentaje'].sum()
        x2=list(df_2.groupby(['respuesta']).sum()['porcentaje'])
        x2=[i*100/x_tot2 for i in x2]
        texto2=[round(i,2) for i in x2]
        total_labels2 = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y2, texto2)]

        df_3 = df.loc[(df.pregunta=='opinion circulacion')]
        y3 = list(df_3.respuesta.unique())
        x_tot3 = df_3['porcentaje'].sum()
        x3=list(df_3.groupby(['respuesta']).sum()['porcentaje'])
        x3=[i*100/x_tot3 for i in x3]
        texto3=[round(i,2) for i in x3]
        total_labels3 = [{"y":y,"x":total*1.1,"text": str(total)+'%', "showarrow": False} for y, total in zip(y3, texto3)]

    with g_1.batch_update():
        g_1.data[0].x = x1
        g_1.data[0].y = y1
        g_1.layout.xaxis.title = ' '
        g_1.layout.yaxis.title = ''
        g_1.layout.annotations = total_labels
    with g_2.batch_update():
        g_2.data[0].y = y2
        g_2.data[0].x = x2
        g_2.layout.xaxis.title = ' '
        g_2.layout.yaxis.title = ''
        g_2.layout.annotations = total_labels2
    with g_3.batch_update():
        g_3.data[0].x = x3
        g_3.data[0].y = y3
        g_3.layout.xaxis.title = ' '
        g_3.layout.yaxis.title = ''
        g_3.layout.annotations = total_labels3
edades_buttons.observe(response, names="value")
genero_buttons.observe(response, names="value")

In [ ]:
widgets.VBox([container,
              g_1,
             g_2,
             g_3])